<font size=14>[**Xarray**](http://xarray.pydata.org/en/stable/)：**多维数组及数据集** </font>

<div align=center><img src="pics/库层.png" width=80%></div>

---

<img src="pics/xarray/Xarray-logo.png" align="right" width=40%>

**Xarray（以前是xray）是一个开源项目和Python包，它使 <font color=Blue>标签多维数组操作 </font>变得简单、高效和有趣！**

<font size=4>

- 多维数组别称：tensors。应用领域广泛：物理、天文、地球科学、生物信息、工程、金融、深度学习……<br/>
- Numpy提供了处理原始N-维数组的数据结构和API，但是现实世界中数据集不单单包含数字，他们有对应时空位置的标签。<br />Xarray在原始的 *类NumPy数组* 的维度Dimensions、坐标Coordinate和属性attributes的结构基础上引入<mark>**标签Label**</mark>，这带来更加直观、更简洁、更不易出错的开发体验。<br/>
- Xarray受到了专注标签表格数据处理流行库Pandas的启发,包含一个大型的、不断增长的领域无关函数库，用于对上述数据结构的高级分析和可视化。<br/>    
- 它特别适合于处理netCDF文件，netCDF文件是xarray数据模型的来源，并与dask紧密集成用于并行计算。<br/>    
- 气象数据大都是多维数组，经度、纬度、高度（气压）、时间、要素，五个“维度”<br/>
- 只是受限于认知方式、技术支持，绝大多数从二维去认知
</font>

---

# 术语

**label**
标签，不仅仅是个“列名”那么简单。可以——<br/>
- 以Python字典的形式跟踪任意元数据：x.attrs
- 通过变量名操作多维数组，比如：x.sum('time')
- <mark>**通过标签而不是位置选中值：x.loc['2014-01-01'] 或 x.sel(time='2014-01-01')**</mark>再也不用数第几层、第几个时次了
- 基于维度名称而不是形状在多维数组上进行数学运算矢量化
- 灵活的 groupby split-apply-combine 操作：x.groupby('time.dayofyear').mean()
- 基于坐标标签的数据库（例如 alignment），可以平滑地处理缺失值：x, y = xr.align(x, y, join='outer')

**dimension：**
在数学中，数据的维数松散地表示为数据的自由度。尺寸轴是所有点的集合，其中除一个自由度外，所有自由度都是固定的。我们可以认为每个维度轴都有一个名称，例如“x维度”。在xarray中，DataArray对象的维度是其命名的维度轴，第i个维度的名称是arr.dims[i]。如果创建的数组没有维度名称，则默认维度名称为dim_0、dim_1，依此类推。

**coordinate：**
为另一个DataArray的维度或维度集添加标签的数组。在通常的一维情况下，坐标数组的值可以松散地看作是沿维度的记号标签。坐标数组有两种类型：尺寸坐标和非尺寸坐标（见下文）。可以从arr.coords[x]检索名为x的坐标。一个DataArray可以有比维度更多的坐标，因为一个维度可以被多个坐标数组标记。但是，只能将一个坐标数组
指定为特定标注的标注坐标数组。

**index：**
索引是一种为有效地选择和切片关联数组而优化的数据结构。Xarray为维度坐标创建索引，这样沿着维度的操作速度很快，而非维度坐标则不会被索引。

**Variable：**
一种类似NetCDF的变量，由描述单个数组的维度、数据和属性组成。变量与numpy数组在功能上的主要区别在于对变量的数值运算实现了按维度名称的数组广播。每个DataArray都有一个可以通过arr.variable访问的基础变量。

# **核心数据结构**
- xarray 有两种基础的数据结构:

  - `DataArray`, 存储单多维变量和它的坐标
  - `Dataset`, 存储多变量，一般变量之间共享同样的坐标
  
<img src="pics/xarray-data-structures.png" align="center" width="50%">

---

## 从netCDF文件中载入数据
- NetCDF (network Common Data Form) 是存储多维数据的格式。[了解更多](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#whatisit)
- NetCDF 自描述，包含数据的信息和必要的元数据例如使用的坐标、数据的属性。
- NetCDF 在地球科学界得到了广泛的应用。
- Xarray 的接口主要基于netCDF数据模型。

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# 打开ECMWF EAR5月平均数据，文件格式为netCDF4
ds = xr.open_dataset("data/ERA5monthlyAveraged.nc")#, engine="netcdf4")
#xr.open_mfdataset('my/files/*.nc', parallel=True)
# Xarray的描述（HTML形式）
ds

<img src="pics/xarray/dataset-diagram.png" width=700/>

在本例中，将温度和降水量称为“数据变量”，将所有其他数组称为“坐标变量”，这是很自然的，因为它们标记了沿维度的点

In [ ]:
#文本形式呈现Xarray的描述信息
xr.set_options(display_style="text")
ds

In [ ]:
#netCDF 形式信息
ds.info()

## ` Dataset `

- Xarray的 `Dataset` 可以看做是标签数组（`DataArrays`）维度对齐后的类字典容器。
- 异构数据，不同类型，甚至不同维度数。
- 除了数据集本身类似dict的接口之外，它还可以用来访问`Dataset`中的任何`DataArray`。
- Dataset可以简单的理解为由多个DataArray组成的集合，具有以下关键属性：

    
|  属性       | 描述                                                                                  |
| ----------- | -------------------------------------------------------------------------------------|
| `data_vars` | 与数据变量相对应的`DataArray`对象的顺序字典OrderedDict。                                |
| `dims`      | 从维度名称到每个维度固定长度的字典映射（例如： {`lat`: 6, `lon`: 6, `time`: 8}）。        |
| `coords`    | 一种类似dict的数组（坐标）容器，用于标记每个点（例如，数字、datetime对象或字符串的一维数组  |
| `attrs`     | OrderedDict保存与数据集相关的任意元数据。                                               |
| `var_name`  | 获取变量                                               |  


In [ ]:
# Dataset当中的变量variable
ds.data_vars

In [ ]:
# Dataset的全局属性
ds.attrs

In [ ]:
# dataset坐标
ds.coords#['time']#.shape

In [ ]:
ds.keys

In [ ]:
ds.t.longitude

In [ ]:
# 变量的选择方式 1   dict-like
ds["t"]

In [ ]:
# 变量的选择方式 2   dict-like
ds.t

In [ ]:
# 删除变量
ds.drop_vars("r")

In [ ]:
# 删除维度
ds.drop_dims("time")

In [ ]:
# 重命名变量。注意：需要重新赋值
ds = ds.rename({"t": "temp", "z": "hgt"})

In [ ]:
ds

In [ ]:
# 分配/追加新变量
ds=ds.assign(tempC=ds.temp-273.15)
ds

100°E，850hPa纬向平均值 随时间的变化

In [ ]:
plt.plot((ds.tempC.sel(level=850,longitude=100.0)).mean("latitude"))
# (ds.tempC.sel(level=850,longitude=100.0)).mean("latitude").plot.line()

2011年1月，850hPa上的，100-120°E均值随纬度的变化

In [ ]:
(ds.tempC.sel(level=850,time='2011-01-01',longitude=np.arange(100,120,0.25))).groupby('latitude').mean(dim="longitude").plot()

## **DataArray**
- 可以看作是pandas.Series的多维形式
- 一个带有标签的多维数组，它有如下几个重要的属性


| 属  性    | 描述                                                        |
| --------- | ---------------------------------------------------------- |
| `data`    | 用`numpy.ndarray` 或 `dask.array`承载数值                   |
| `dims`    | 获取维度的名字，如(`x`, `y`, `z`) (`lat`, `lon`, `time`)。   |
| `coords`  | 获取一个类似于字典的结果，里面包含各个坐标                     |
| `attrs`   | 获取原始数据的属性，比如变量的名字、单位等                     |
| `name`    | 数组的任意名称                                              |


<img src="pics/xarray/dataset-diagram.png" width=700/>

↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑  最左边这个“cube” .ie. ds.t

<img src="pics/xarray/多维数组-T.png" width=700/>

In [ ]:
da=ds.t
da

In [ ]:
# 获取数值，类型为ndarray
da.values#da.data      type(ds.air.data)

In [ ]:
# 获取维度
da.dims

In [ ]:
# 获取坐标
da.coords

In [ ]:
# 获取属性
da.attrs

In [ ]:
# 获取变量名
da.name

# 切片与索引

## 转Numpy-位置索引

In [ ]:
t = ds["t"].data  # numpy array
t

In [ ]:
t.shape

In [ ]:
# 输出空间（层次、纬度、经度）特定点的时间序列图
# 物理/气象意义上：月平均温度随时间变化
plt.plot(t[:, 6, 120, 200]-273.15)    #数学意义上 1st维度从头到尾， 2nd维度第7个  ，3rd维度第121个    ，4th纬度第201个
                               #物理/气象意义上，2011.01-2020.12，第7层1000hPa   ，70-120*0.25=40°N ，70＋200*0.25=120°E

## xarray索引

| 维度查找    | 索引查找  |  DataArray语法                                |  Dataset 语法                       |
| ----------- | --------- |---------------------------------------------- | ----------------------------------- |
| `位置`      | 使用整数  |  da[:, 0]                                     | 无法访问                            |
| `位置`      | 使用标签  |  da.loc[:, "IA"]                              | 无法访问                            |
| `名称`      | 使用整数  |  **da.isel(space=0)** 或da[dict(space=0)]          | **ds.isel(space=0)** 或ds[dict(space=0)] |
| `名称`      | 使用标签  |  **da.sel(space='IA')** 或da.loc[dict(space='IA')] | **ds.sel(space='IA')** 或ds.loc[dict(space='IA')]|


<div class="section" id="indexing-rules">
<span id="id12"></span><h3>索引规则<a class="headerlink" href="#indexing-rules" title="Permalink to this headline"></a></h3>
<p>这里我们描述xarray用于矢量化索引的完整规则。注意，这是为了解释：为了提高效率和支持各种后端，实际实现是不同的。</p>
<ol class="arabic simple" start="0">
<li><p>（仅适用于基于标签的索引。）从相应的pandas.Index沿每个维度查找位置索引。 <a class="reference external" href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html#pandas.Index" title="(in pandas v1.2.5)"><code class="xref py py-class docutils literal notranslate"><span class="pre">pandas.Index</span></code></a>.</p></li>
<li><p>完整的切片对象 <code class="docutils literal notranslate"><span class="pre">:</span></code> 在没有索引器的情况下为每个维度插入。</p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">切片</span></code> 对象转换为数组, 由
<code class="docutils literal notranslate"><span class="pre">np.arange(*slice.indices(...))</span></code>给出.</p></li>
<li><p>假设没有维度的数组索引器的维度名称，例如np.ndarray和list，
<code class="docutils literal notranslate"><span class="pre">np.ndarray</span></code> and <code class="docutils literal notranslate"><span class="pre">list</span></code>, 来自要索引的维度。例如, <code class="docutils literal notranslate"><span class="pre">v.isel(x=[0,</span> <span class="pre">1])</span></code> 被理解为
<code class="docutils literal notranslate"><span class="pre">v.isel(x=xr.DataArray([0,</span> <span class="pre">1],</span> <span class="pre">dims=['x']))</span></code>.</p></li>
<li><p>对于 <code class="docutils literal notranslate"><span class="pre">Dataset</span></code> 或 <code class="docutils literal notranslate"><span class="pre">DataArray</span></code> （数组及其坐标）中的每个变量:</p>
<ol class="loweralpha simple">
<li><p>基于维度名称广播所有相关索引器（有关详细信息，请参见按维度名称广播）。</p></li>
<li><p>使用NumPy的高级索引规则，由广播索引器对下划线数组进行索引。</p></li>
</ol>
</li>
<li><p>如果任何索引器DataArray具有坐标，但不存在同名坐标，请将它们附加到索引对象。</p></li>
</ol>
</div>

### 位置索引

<img src="pics/xarray/多维数组-T-time.png" width=100%/>

In [ ]:
# 一共有4个维度，4个axis轴，顺序为time、level、latitude、longitude
(ds.t[ :, 2, 120, 200] - 273.15).plot() #从轴1（time）选取所有时间，轴2（level）选取第3个，轴3（latitude）选取第120个,轴4（longitude）选取第200个

<img src="pics/xarray/多维数组-T-time0+500.png" width=100%/>

In [ ]:
# 得到二维数据
# 一共有4个维度，4个axis轴，顺序为time、level、latitude、longitude
ds.t[0,2] #从轴1（time）选取第一个时间2011年1月，轴2（level）选取第3个500hPa

In [ ]:
# 绘制等值线分析
#(ds.t[0,6,:,:]-273.15).plot.contour(levels=range(-30,30,2))
# 绘制等值线填色分析
(ds.t[0,6,:,:]-273.15).plot(levels=range(-30,30,1))

### 维度索引

In [ ]:
(ds.t.isel(time=0,level=6,latitude=40)-273.15).plot()

<div class="alert alert-block alert-warning">
isel()是index select的意思：按照index进行选择。<br/>
    所以这里的latitude=40，其实是第41个索引，也就是70-40*0.25＝60°N<br/>
    本质上还是和Numpy的index索引一样
</div>

In [ ]:
(ds.t.loc[dict(time="2011-01", level=1000, latitude=40)]-273.15).plot()

In [ ]:
(ds.t.sel(time="2011-01", level=1000, latitude=40)-273.15).plot()

<div class="alert alert-block alert-warning">
    sel()是select的意思：按照标签进行选择<br/>
    用户已知经度、纬度、层次、时间，可以直接快速找到 `最直观` 的coords对应的数据（要哪找哪）
    </div>

In [ ]:
# 维度切片
ds.sel(time=slice("2019-05", "2020-07"))

---
与numpy和pandas一样，xarray支持以矢量化方式同时索引许多数组元素：

### 临近索引

In [ ]:
ds.sel(latitude=38.5, longitude=115.98, method="nearest")

In [ ]:
# 生成截面坐标 generate a coordinates for a transect of points
lat_points = xr.DataArray([10, 20, 30], dims="points")
lon_points = xr.DataArray([100, 110, 120], dims="points")
lat_points

In [ ]:
lon_points

In [ ]:
# 沿着截面进行临近选择 nearest neighbor selection along the transect
(ds.t.sel(level=1000,latitude=lat_points, longitude=lon_points)-273.15).plot.line(x='time')

### 使用`where()`索引


In [ ]:
# 查找高温值
ds.t.where(ds.t>273.15+35).sel(level=1000,time="2019-07").plot()
# 替换空值 Let's replace the missing values (nan) with some placeholder

# 插值

xarray提供了灵活的插值例程，它与我们的索引具有类似的接口。interp需要安装scipy。

## 标量和一维插值
对DataArray进行插值 与 对它进行标签检索很像。

In [ ]:
#2020年9月高度场
ds.z.sel(time="2020-09-01",level=500).plot.contour(levels=range(50000,60000,400))
#2020年10月高度场
ds.z.sel(time="2020-10-01",level=500).plot.contour(levels=range(50000,60000,400),colors='k')

In [ ]:
#内插得到9月15日的高度场
ds.z.sel(level=500).interp(time="2020-09-15").plot.contour(levels=range(50000,60000,400))
#内插得到9月2日的高度场
#ds.z.sel(level=500).interp(time="2020-09-02").plot.contour(levels=range(50000,60000,400))

In [ ]:
#内插得到9月15日的高度场，采用临近插值【只是选定，而非“插值”】
'''
DataArray.interp(coords=None, method='linear', assume_sorted=False, kwargs=None, **coords_kwargs)
    {“linear”, “nearest”} for multidimensional array,
    {“linear”, “nearest”, “zero”, “slinear”, “quadratic”, “cubic”} for 1-dimensional array.
'''
ds.z.sel(level=500).interp(time="2020-09-15", method="nearest").plot.contour(levels=range(50000,60000,400))

In [ ]:
#插值得到2020年10月的 *800hPa*温度场
ds.t.sel(time="2020-10").interp(level=800).plot()

## 多维插值
与 `sel()` 一样，interp()接受多个坐标。在这种情况下，执行多维插值

In [ ]:
ds.t.interp( time="2020-09-15", level=800).plot()

## 高级插值
`interp()`接受类似于`sel()`的`DataArray`，这使我们能够进行更高级的插值。根据传递给`interp()`的新坐标的尺寸，确定结果的尺寸。
例如，如果要沿特定维度内插二维数组，如下图所示，可以传递两个具有公共维度的一维数组数据作为新坐标。

In [ ]:
# 索引2020年9月1日的各层温度场
t=ds.t.sel(time='2020-09-01')-273.15
t

In [ ]:
#找到三个经纬度点上的数据，分别取值
lons = xr.DataArray([100,110,120],dims='z')
lats = xr.DataArray([40,50,60],dims='z')

t.sel( longitude =lons, latitude=lats).plot()

<img src="pics/xarray/advanced_selection_interpolation.svg" align="center" width="45%">

In [ ]:
lons = xr.DataArray([100.1,111.1,112.1])
lats = xr.DataArray([40.1,41.1,42.1])
#内插得到 非原有网格上 经纬度点的数据
t.interp( longitude =lons, latitude=lats).plot.contourf()

In [ ]:
#沿着任意经纬度点进行插值
import numpy as np
lons = xr.DataArray(np.arange(100.1,120.1,0.1))
lats = xr.DataArray(np.arange(40.1,60.1,0.1))
fig, ax = plt.subplots(figsize=(10, 4))
dsi = ds.r.sel(time='2020-09-01').interp( longitude =lons, latitude=lats)
dsi.plot.contourf(ax=ax)
ax.invert_yaxis()

In [ ]:
# 水平插值  提高“精度”
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
fig, axes = plt.subplots(ncols=2, figsize=(10, 4))

t.sel(level=850).plot(ax=axes[0],levels=range(0,25,2))
axes[0].set_title("原始数据")
axes[0].set_xlim(70,100)
axes[0].set_ylim(28,40)

new_lon = np.linspace(ds.longitude[0], ds.longitude[-1], ds.dims["longitude"] * 4) #！！！网格加密4倍
new_lat = np.linspace(ds.latitude[0], ds.latitude[-1], ds.dims["latitude"] * 4)    #！！！网格加密4倍
dsi = t.sel(level=850).interp(latitude=new_lat, longitude=new_lon)
dsi.plot(ax=axes[1],levels=range(0,25,2))
axes[1].set_title("插值数据")
axes[1].set_xlim(70,100)
axes[1].set_ylim(28,40)

*高级可以用来重新映射数据到新的坐标。(参见官方示例……)*<br/>
[<img src="pics/xarray/interpolation_sample4.png" align="center" width="40%">](http://xarray.pydata.org/en/stable/user-guide/interpolation.html?highlight=interpolation)

```python
# new coordinate
In [53]: x = np.linspace(240, 300, 100)
In [54]: z = np.linspace(20, 70, 100)
# relation between new and original coordinates
In [55]: lat = xr.DataArray(z, dims=["z"], coords={"z": z})
In [56]: lon = xr.DataArray(
   ....:     (x[:, np.newaxis] - 270) / np.cos(z * np.pi / 180) + 270,
   ....:     dims=["x", "z"],
   ....:     coords={"x": x, "z": z},
   ....: )
   ....: 
In [57]: fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
In [58]: ds.air.plot(ax=axes[0])
Out[58]: <matplotlib.collections.QuadMesh at 0x7f383f100f70>
# draw the new coordinate on the original coordinates.
In [59]: for idx in [0, 33, 66, 99]:
   ....:     axes[0].plot(lon.isel(x=idx), lat, "--k")
   ....: 
In [60]: for idx in [0, 33, 66, 99]:
   ....:     axes[0].plot(*xr.broadcast(lon.isel(z=idx), lat.isel(z=idx)), "--k")
   ....: 
In [61]: axes[0].set_title("Raw data")
Out[61]: Text(0.5, 1.0, 'Raw data')
In [62]: dsi = ds.interp(lon=lon, lat=lat)
In [63]: dsi.air.plot(ax=axes[1])
Out[63]: <matplotlib.collections.QuadMesh at 0x7f383ee6ba30>
In [64]: axes[1].set_title("Remapped data")
Out[64]: Text(0.5, 1.0, 'Remapped data')
```

# 计算Computation

## 简单统计
聚合方法已更新为采用dim参数而不是axis。这就为沿特定维度应用的聚合方法提供了非常直观的语法：

In [ ]:
ds.w.sel(time="2020-09-01").sum(dim="level").plot()

In [ ]:
t.sel(level=850).sum()#求和

In [ ]:
t.sel(level=850).mean()#求均值

In [ ]:
t.sel(level=850).min()#最小值，最大值max()

In [ ]:
t.sel(level=850).std()#标准差

In [ ]:
tSeries = ds.t.sel(level=1000,latitude=40,longitude=120)-273.15
#求滑动平均
a = tSeries.rolling(time=9, center=True).mean() 
plt.plot(a)
plt.plot(tSeries)

*DataArray可以进行权重计算` DataArray.weighted() `*

## *求导*


In [ ]:
import numpy as np
# 我们将添加一个具有适当属性的梯度场
ds["dTdx"] = ds.t.differentiate("longitude") / 110e3 / np.cos(ds.latitude * np.pi / 180)
ds["dTdy"] = ds.t.differentiate("latitude") / 105e3
ds.dTdx.attrs = {"long_name": "$∂T/∂x$", "units": "°C/m"}
ds.dTdy.attrs = {"long_name": "$∂T/∂y$", "units": "°C/m"}

ds

In [ ]:
t.sel(level=850).differentiate("latitude").plot(levels=range(-3,2,1))

In [ ]:
t.sel(level=850).plot(levels=range(-30,20,1))

In [ ]:
re = np.gradient(t.sel(level=850))
grad = np.sqrt(np.sum(np.square(re), axis=0))
plt.contourf(grad,levels=np.arange(0.1,3,0.1))

## *缺测值*
像Pandas一样 `isnull(), notnull(), count(), dropna(), fillna(), ffill(),bfill()` 方法用于处理缺测数据

与Pandas一样，同样使用`np.nan`（not-a-number）代表缺测值。

# GroupBy:拆分-应用-结合
xarray使用与pandas相同的API支持“group by”操作，以实现split-apply-combine策略：

* 将数据拆分为多个独立组。
* 对每组应用一些函数。
* 将组合并回单个数据对象。

分组操作对`Dataset`和`DataArray`对象都起作用。尽管最近实现了对多维变量分组的支持，但大多数示例都关注于通过单个一维变量分组。请注意，对于一维数据，依赖pandas实现相同的管道通常更快。

In [ ]:
t.groupby('level')

In [ ]:
t.groupby('level').groups

In [ ]:
list(t.groupby('level'))

In [ ]:
t.groupby('level')[850]

In [ ]:
#对时间求平均
ave = (ds.z.groupby('level').mean(dim='time')).sel(level=500)
#std = (ds.z.groupby('level').std(dim='time')).sel(level=500)
fig, axes = plt.subplots(ncols=3, figsize=(10, 4))

#画出平均场
cs0 = axes[0].contour(ave[::-1],levels=range(50000,60000,400))
axes[0].clabel(cs0,cs0.levels,inline=True)

#画出2020年9月场
cs1 = axes[1].contour(ds.z.sel(level=500,time="2020-09-01")[::-1],
                levels=range(50000,60000,400))
axes[1].clabel(cs1,cs1.levels,inline=True)

# 画出距平场
cs2 = axes[2].contour((ds.z.sel(level=500,time="2020-09-01")-ave)[::-1])
axes[2].clabel(cs2,cs2.levels,inline=True)

In [ ]:
# 季节分组
ds.groupby("time.season")

In [ ]:
# 季节平均
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean

In [ ]:
seasonal_mean.t.sel(level=1000).plot(col="season",robust=True)

In [ ]:
# resample到年平均
ds.resample(time="Y").mean()
# resample到两个月
# ds.r.resample(time="2MS").mean()

# 输出文件

输出nc文件

In [ ]:
ave.to_netcdf('data/ave.nc')

In [ ]:
ds1 =xr.open_dataset('data/ave.nc')
ds1

<font color=#FF0000>**通过cfgrib引擎，调用eccodes扩展库/插件（生成idx文件）解析GRIB数据，读取后存储在xr.DataArray数据结构中**</font>

In [ ]:
# 打开grib文件
ds_grib = xr.open_dataset("data/Z_NAFP_C_BABJ_20210623000000_P_CNPC-GRAPES-RMFS-FCSTER-02400.grib2"
                          ,engine="cfgrib"
                          ,filter_by_keys={'stepType': 'instant','typeOfLevel': 'isobaricInhPa'})
ds_grib